In [1]:
# import packages
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns 
from sklearn.metrics import pairwise_distances
import statsmodels.api as sm

## Read the data

In [2]:
# read sp-level abundance table
path = '/Users/katerynapantiukh/Documents/1MyDisk/PhD/!MAIN_data/'
#path = '/Users/ketpantuh/Documents/1MyDisk/PhD/!MAIN_data/' # home

abud = pd.read_csv(str(path)+'bgi_abud_tables/bgi_assembly_v1_SpLevel_RelAbund.csv')
# skip first row
abud = abud.iloc[1:]
abud.set_index('rep_MAG_ID', inplace=True)
abud = abud.T
abud.reset_index(inplace=True)

In [3]:
# read the phenotypes
import os
files = os.listdir(str(path)+'pheno/')
# remove hidden files
files = [file for file in files if not file.startswith('.')]
# remove first 5 characters from each file name
files = [file[6:] for file in files]
ph_list = [file[:-4] for file in files]

In [4]:
# drugs description
ph_names = pd.read_excel(str(path)+'pheno_description/phenotype_names.xlsx')
ph_names.head(2)

,code,name,type
0,B18,Chronic viral hepatitis,disease
1,B37,Candidiasis,disease


## Data preparation

In [5]:
# filter out sp with prevalence <1%
abud = abud.loc[:, abud.astype(bool).sum(axis=0) >= 21]

# perform CLR transformation for columns starting from H
abud.iloc[:, 1:] = abud.iloc[:, 1:].apply(lambda x: x/x.sum(), axis=1)
abud.to_csv('bgi_assembly_v1_SpLevel_RelAbund_CLR_1prev.csv')

# write stats
fspNumb = len(abud.columns)
perc = (len(abud)-fspNumb)/len(abud)*100

# Bonferoni corrected significance level
alpha = 0.05 / fspNumb

print(f"Initial bacteria species count is {len(abud)}.")
print(f"Final sp. number after filtering is {fspNumb}.")
print(f"{str(perc)[0:5]}% of bacteria species were filtered out.")
print(f"Significance level after Bonferoni correction is {alpha}.")

Initial bacteria species count is 2101.
Final sp. number after filtering is 1595.
24.08% of bacteria species were filtered out.
Significance level after Bonferoni correction is 3.134796238244514e-05.


In [6]:
des = ph_names[ph_names['type'] == 'disease']['code'].to_list()
ph_des_sel = []

for ph_name in des:
    # read phenotype
    ph = pd.read_csv(str(path)+'/pheno/Pheno_'+ph_name+'.csv')
    if ph[ph_name].sum() > 100:
        # add value to list
        ph_des_sel.append(ph_name)

perc2 = (len(des)-len(ph_des_sel))/len(des)*100

print(f"Initial number of disease phenotype is {len(des)}.")
print(f"Number of phenotype with more then 100 cases is {len(ph_des_sel)}.")
print(f"{str(perc2)[0:5]}% of phenotypes were filtered out.")

Initial number of disease phenotype is 66.
Number of phenotype with more then 100 cases is 33.
50.0% of phenotypes were filtered out.


In [7]:
dr = ph_names[ph_names['type'] != 'disease']['code'].to_list()
ph_dr_sel = []

for ph_name in dr:
    # read phenotype
    ph = pd.read_csv(str(path)+'/pheno/Pheno_'+ph_name+'.csv')
    if ph[ph_name].sum() > 100:
        # add value to list
        ph_dr_sel.append(ph_name)

perc3 = (len(dr)-len(ph_dr_sel))/len(dr)*100

print(f"Initial number of drugs phenotype is {len(dr)}.")
print(f"Number of phenotype with more then 100 cases is {len(ph_dr_sel)}.")
print(f"{str(perc3)[0:5]}% of phenotypes were filtered out.")

Initial number of drugs phenotype is 132.
Number of phenotype with more then 100 cases is 1.
99.24% of phenotypes were filtered out.


## Start loop correlation analysis through the phenotypes

In [8]:
# one phenotype test
ph_name = 'D50'
# Define your covariates
covariates = ['Age_at_MBsample', 'BMI', 'gender']

pheno = pd.read_csv(str(path)+'pheno/Pheno_'+str(ph_name)+'.csv')
mrg = abud.merge(pheno, left_on='index', right_on='vkood', how='inner')

# perform CLR transformation for columns with bacteria species
tc = len(mrg.columns)-5

# fill NA values with mean
mrg['BMI'].fillna(mrg['BMI'].mean(), inplace=True)
mrg['gender'].fillna(mrg['gender'].mean(), inplace=True)
mrg['Age_at_MBsample'].fillna(mrg['Age_at_MBsample'].mean(), inplace=True)

# create a df with results for each phenotype
df = pd.DataFrame(columns=['bacteria', 'p-value', 'betta'])

n = len(mrg.columns)-5
bac = mrg.columns[1:n]
len(bac)

for b in bac:
    # Create a design matrix by adding the covariates to the model
    X = mrg[covariates + [ph_name]]
    X = sm.add_constant(X)

    y = mrg[b]
    model = sm.OLS(y, X).fit()
    if model.pvalues[ph_name] < alpha:
                print(b)
                print(model.pvalues[ph_name])  
                print(model.params[ph_name])
                print('-----------------')


H1531_CALXSV01_sp944390945
1.9477281427590836e-05
2.2191754904054234e-05
-----------------
H0862_Phil1_sp004558525
8.346134583067753e-06
0.0005307679065972345
-----------------
H0583_Absicoccus_sp000434355
2.861282808867806e-05
0.0001543716513238516
-----------------


In [9]:
# phenotypes with more then 100 cases
nMore = []
# number of phenotypes with less then 100 cases
nLess = []

# Define your covariates
covariates = ['Age_at_MBsample', 'BMI', 'gender']
# create a new df for all correlation table
allCor = pd.DataFrame(columns=['pheno','bacteria', 'p-value', 'betta'])
# create a new df for aggregated results
res = pd.DataFrame(columns=['phenotype', 'ass_found', 'ass_bact','cases','controls'])

for ph_name in ph_des_sel:
    #reads phenotype data
    pheno = pd.read_csv(str(path)+'pheno/Pheno_'+str(ph_name)+'.csv')
    mrg = abud.merge(pheno, left_on='index', right_on='vkood', how='inner')

    # perform CLR transformation for columns with bacteria species
    tc = len(mrg.columns)-5
    #mrg.iloc[:, 1:tc] = mrg.iloc[:, 1:tc].apply(lambda x: x/x.sum(), axis=1)

    if len(mrg[mrg[ph_name] == 1]) <100:
        print("There are not enought samples for phenotype {}. Only {} cases found".format(ph_name, len(mrg[mrg[ph_name] == 1])))
        nLess.append(ph_name)
        print('---')
    else:
        print("WOW! It is enought samples for phenotype {}. We found {} cases!".format(ph_name, len(mrg[mrg[ph_name] == 1])))
        nMore.append(ph_name)
        print('...')

        # fill NA values with mean
        mrg['BMI'].fillna(mrg['BMI'].mean(), inplace=True)
        mrg['gender'].fillna(mrg['gender'].mean(), inplace=True)
        mrg['Age_at_MBsample'].fillna(mrg['Age_at_MBsample'].mean(), inplace=True)

        # create a df with results for each phenotype
        df = pd.DataFrame(columns=['bacteria', 'p-value', 'betta'])

        n = len(mrg.columns)-5
        bac = mrg.columns[1:n]
        len(bac)

        for b in bac:
            # Create a design matrix by adding the covariates to the model
            X = mrg[covariates + [ph_name]]
            X = sm.add_constant(X)

            y = mrg[b]

            # Fit the linear regression model
            model = sm.OLS(y, X).fit()
            # Fit the logistic regression model
            #model = sm.Logit(y, X).fit(method='bfgs', maxiter=1000) 
            #model = sm.Probit(y, X).fit(method='bfgs', maxiter=1000)

            if model.pvalues[ph_name] < alpha:
                print(b)
                #print(model.pvalues[ph_name])   

                # fill out separete table for phenotype
                rows_to_append = []
                new_row = {'bacteria': b, 'p-value': model.pvalues[ph_name], 'betta': model.params[ph_name]}
                rows_to_append.append(new_row)
                new_data = pd.DataFrame(rows_to_append)
                df = pd.concat([df, new_data], ignore_index=True)

                # fill out AllCor table
                rows_to_append = []
                new_row = {'pheno': ph_name, 'bacteria': b, 'p-value': model.pvalues[ph_name], 'betta': model.params[ph_name]}
                rows_to_append.append(new_row)
                new_data = pd.DataFrame(rows_to_append)
                allCor = pd.concat([allCor, new_data], ignore_index=True)

            df.sort_values(by=['p-value'], inplace=True)
            df.to_csv('results/sm-Probit/Pval_'+str(ph_name)+'.csv', index=False)     
        
        if df.empty:
            print('There are no significant correlations')
            new_row = {'phenotype': ph_name, 'ass_found': 'no','ass_bact': '-','cases': len(mrg[mrg[ph_name] == 1]),'controls':len(mrg[mrg[ph_name] == 0])}
        else:
            bct = ', '.join(df['bacteria'].to_list())
            new_row = {'phenotype': ph_name, 'ass_found': 'yes','ass_bact': bct,'cases': len(mrg[mrg[ph_name] == 1]),'controls':len(mrg[mrg[ph_name] == 0])}

        rows_to_append = []
        rows_to_append.append(new_row)

        new_data = pd.DataFrame(rows_to_append)
        res = pd.concat([res, new_data], ignore_index=True)

        print('---')

# Write the results!
allCor = allCor.merge(ph_names, left_on='pheno', right_on='code', how='left')
allCor = allCor[['pheno','name','bacteria','p-value', 'betta']]
allCor.to_excel('results/allCorrelation_table_v2.xlsx', index=False) 

res = res.merge(ph_names, left_on='phenotype', right_on='code', how='left')
res = res[['code','name','ass_found','ass_bact','cases','controls']].sort_values(by='ass_found', ascending=False)
res.to_excel('results/aggrCorrelation_table_v2.xlsx', index=False) 


WOW! It is enought samples for phenotype B37. We found 407 cases!
...
There are no significant correlations
---
WOW! It is enought samples for phenotype D50. We found 255 cases!
...
H1531_CALXSV01_sp944390945
H0862_Phil1_sp004558525
H0583_Absicoccus_sp000434355
---
WOW! It is enought samples for phenotype E55. We found 102 cases!
...
H2215_Ventrimonas_sp900540335
H0885_Borkfalkia_excrementipullorum
H1238_Enterocloster_clostridioformis
H0653_Aphodousia_sp900544345
H1619_Oscillospiraceae_undG
---
WOW! It is enought samples for phenotype E78. We found 417 cases!
...
There are no significant correlations
---
WOW! It is enought samples for phenotype F32. We found 392 cases!
...
There are no significant correlations
---
WOW! It is enought samples for phenotype F33. We found 177 cases!
...
H1506_Hungatella_effluvii
H0478_Desulfovibrio_faecigallinarum
H0722_Bacteroides_faecis
H2126_Alistipes_undS
---
WOW! It is enought samples for phenotype F41. We found 287 cases!
...
H2123_Paraprevotella_sp9

In [10]:
%%capture captured_output
k = len(res[res['ass_found'] == 'yes'])

# print current date
from datetime import datetime

current_date = datetime.now()
print(current_date, '\n')

print(f"Bacteria filter: sp. with prevalence <1%")
print(f"Initial bacteria species count is {len(abud)}.")
print(f"Final sp. number after filtering is {fspNumb}.")
print(f"{str(perc)[0:5]}% of bacteria species were filtered out.\n")

print(f"Data preparation: SLR transformation")
print(f"Significance level: Bonferoni correction")
print(f"Significance level after is {alpha}. \n")

print(f"Phenotype filter: filter out phenotypes with less then 100 cases")
print(f"It was analised 66 phenotypes")
print(f"Fot {len(nMore)} phenotypes with enought cases")
print(f"Among them {k} phenotypes had {len(allCor)} associations \n")

print(f"Phenotypes with associations:")
a = ', '.join(res[res['ass_found'] == 'yes']['name'].to_list())
print(a, '\n')

print(f"Phenotypes without associations:")
b = ', '.join(res[res['ass_found'] != 'yes']['name'].to_list())
print(b)

In [12]:
with open('results/log.csv', 'w') as f:
    f.write(captured_output.stdout)